In [3]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense,Activation,Flatten,Conv2D,MaxPooling2D,BatchNormalization,Dropout,AveragePooling2D, Dropout
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import tensorflow as tf
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications import EfficientNetB5,inception_v3,Xception,ResNet101V2,NASNetLarge
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
import cv2
import time
import pandas as pd
from tqdm import tqdm
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import albumentations

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
img_path = 'Unit Test Data/ALIS_UnitTest_2020_Q4/PNG'
all_img_name= os.listdir(img_path)
print(len(all_img_name))

9057


In [14]:
%%time
X_test=[]
a=1
model = load_model('model/0125_2class_9.h5')
for i in all_img_name[:10]:
    start = time.time()
    img= cv2.imread(img_path+'/'+i, 1)
    img = img[250:750, 250:750]
    img= cv2.resize(img, (256,256),interpolation=cv2.INTER_AREA)
    predictions = model.predict_classes(img)
    end = time.time()
    X_test.append(img)
    print("第%d張 : "%a,end='')
    print("%.5f秒"%(end-start))
    a+=1
X_test = np.array(X_test)
X_test =preprocess_input(X_test)
print(X_test.shape)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
第1張 : 1.99223秒

第2張 : 0.11600秒

第3張 : 0.11005秒

第4張 : 0.10708秒

第5張 : 0.10796秒

第6張 : 0.10402秒

第7張 : 0.10903秒

第8張 : 0.10703秒

第9張 : 0.10602秒

第10張 : 0.10703秒

第11張 : 0.10803秒

第12張 : 0.10903秒

第13張 : 0.10902秒

第14張 : 0.10603秒

第15張 : 0.10605秒

第16張 : 0.10699秒

第17張 : 0.10707秒

第18張 : 0.10698秒

第19張 : 0.10802秒

第20張 : 0.10703秒

第21張 : 0.11106秒

第22張 : 0.11198秒

第23張 : 0.1110

In [9]:
%%time
model = load_model('model/0125_2class_9.h5')
predictions = model.predict_classes(X_test) 
print(predictions.shape)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(9057,)
Wall time: 41.9 s


In [10]:
families = ['inclusions', 'non_inclusions']

In [11]:
predicts = pd.DataFrame({'Defect ID':all_img_name,'Prediction':predictions})
predicts = predicts.replace(range(len(families)),families)
predicts = predicts.sort_values(by='Defect ID')

Y_test = pd.read_csv('Unit Test Data/ALIS_UnitTest_2020_Q4/ALIS_UnitTest_2020_Q4.csv')
Y_test = Y_test.sort_values(by='Defect ID')
Index_table = Y_test
Y_test = Y_test.replace(['BLISTER','INCLUSION_FLASHLIGHT','INCLUSION_SDX'],'inclusions')
Y_test = Y_test.replace(['CLUSTEREDPARTICLES', 'DARKSPOT', 'NO_DEFECT','BROKENGLASS',
            'CARRIER', 'REFLECTION', 'SHEETEDGE'],'non_inclusions')

In [12]:
res = pd.merge(Y_test,predicts, on='Defect ID')
res

,Defect ID,Type,Prediction
0,TN02_20191125144955_3.png,inclusions,inclusions
1,TN02_20191125145059_1.png,inclusions,inclusions
2,TN02_20191125145059_2.png,inclusions,inclusions
3,TN02_20191125145411_1.png,inclusions,inclusions
4,TN02_20191125145547_1.png,inclusions,inclusions
...,...,...,...
9052,TN02_20201229013822_2.png,inclusions,inclusions
9053,TN02_20201229013848_1.png,inclusions,inclusions
9054,TN02_20201229053130_1.png,inclusions,inclusions
9055,TN02_20201229054109_2.png,inclusions,inclusions


In [13]:
cf_df = pd.crosstab(res['Type'], res['Prediction'], rownames=['實際值'], colnames=['預測值'])
cf_df['Accuracy']=[cf_df.iloc[0][0]/(cf_df.iloc[0][0]+cf_df.iloc[0][1]),
                   cf_df.iloc[1][1]/(cf_df.iloc[1][0]+cf_df.iloc[1][1])]
cf_df.to_csv('confuse0125_2class_9.csv')
print("Total Accuracy:",
      (cf_df.iloc[0][0] + cf_df.iloc[1][1])/(cf_df.iloc[0][0]+cf_df.iloc[0][1]+cf_df.iloc[1][0]+cf_df.iloc[1][1]))
cf_df

Total Accuracy: 0.9834382245776747


預測值,inclusions,non_inclusions,Accuracy
實際值,,,
inclusions,2965,39,0.987017
non_inclusions,111,5942,0.981662


In [9]:
correct = res[res['Type'] == res['Prediction']]
correct

,Defect ID,Type,Prediction
0,TN02_20191125144955_3.png,inclusions,inclusions
1,TN02_20191125145059_1.png,inclusions,inclusions
2,TN02_20191125145059_2.png,inclusions,inclusions
3,TN02_20191125145411_1.png,inclusions,inclusions
4,TN02_20191125145547_1.png,inclusions,inclusions
...,...,...,...
9052,TN02_20201229013822_2.png,inclusions,inclusions
9053,TN02_20201229013848_1.png,inclusions,inclusions
9054,TN02_20201229053130_1.png,inclusions,inclusions
9055,TN02_20201229054109_2.png,inclusions,inclusions


In [10]:
import shutil
path = 'Result_3'
img_path = 'Unit Test Data/ALIS_UnitTest_2020_Q4/PNG'
for i in range(len(correct)):
    name = correct.iloc[i]['Defect ID']
    folder = correct.iloc[i]['Type']
    print(img_path+"/"+str(name))
    print(path+"/"+str(folder)+"/"+str(name))
    
    shutil.copyfile(img_path+"/"+str(name), path+"/Correct/"+str(folder)+"/"+str(name))


Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125144955_3.png
Result_3/inclusions/TN02_20191125144955_3.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125145059_1.png
Result_3/inclusions/TN02_20191125145059_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125145059_2.png
Result_3/inclusions/TN02_20191125145059_2.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125145411_1.png
Result_3/inclusions/TN02_20191125145411_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125145547_1.png
Result_3/inclusions/TN02_20191125145547_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125172029_98.png
Result_3/inclusions/TN02_20191125172029_98.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125172029_99.png
Result_3/inclusions/TN02_20191125172029_99.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125180050_107.png
Result_3/inclusions/TN02_20191125180050_107.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125180050_54.png
Result_3/inclusions/

Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125200311_13.png
Result_3/inclusions/TN02_20191125200311_13.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125200311_3.png
Result_3/inclusions/TN02_20191125200311_3.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125200654_5.png
Result_3/inclusions/TN02_20191125200654_5.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125200654_6.png
Result_3/inclusions/TN02_20191125200654_6.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125200654_7.png
Result_3/inclusions/TN02_20191125200654_7.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125200901_8.png
Result_3/inclusions/TN02_20191125200901_8.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125201708_6.png
Result_3/inclusions/TN02_20191125201708_6.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125201748_2.png
Result_3/inclusions/TN02_20191125201748_2.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125201748_9.png
Result_3/inclusions/TN02_20

Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125224214_17.png
Result_3/inclusions/TN02_20191125224214_17.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125224246_1.png
Result_3/inclusions/TN02_20191125224246_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125224246_2.png
Result_3/inclusions/TN02_20191125224246_2.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125224246_4.png
Result_3/inclusions/TN02_20191125224246_4.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125224246_5.png
Result_3/inclusions/TN02_20191125224246_5.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125224246_6.png
Result_3/inclusions/TN02_20191125224246_6.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125224246_7.png
Result_3/inclusions/TN02_20191125224246_7.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125224246_8.png
Result_3/inclusions/TN02_20191125224246_8.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20191125224246_9.png
Result_3/inclusions/TN02_20

Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200101021759_4.png
Result_3/non_inclusions/TN02_20200101021759_4.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200101022303_2.png
Result_3/non_inclusions/TN02_20200101022303_2.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200101022444_5.png
Result_3/non_inclusions/TN02_20200101022444_5.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200101025320_3.png
Result_3/non_inclusions/TN02_20200101025320_3.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200101030738_7.png
Result_3/non_inclusions/TN02_20200101030738_7.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200101030738_8.png
Result_3/non_inclusions/TN02_20200101030738_8.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200101031150_2.png
Result_3/inclusions/TN02_20200101031150_2.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200101033846_4.png
Result_3/non_inclusions/TN02_20200101033846_4.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200101034530_6.png
R

Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200104040404_2.png
Result_3/inclusions/TN02_20200104040404_2.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200104041647_8.png
Result_3/inclusions/TN02_20200104041647_8.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200104051423_1.png
Result_3/inclusions/TN02_20200104051423_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200104053535_3.png
Result_3/inclusions/TN02_20200104053535_3.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200104070654_10.png
Result_3/inclusions/TN02_20200104070654_10.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200104073059_3.png
Result_3/inclusions/TN02_20200104073059_3.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200104083057_7.png
Result_3/inclusions/TN02_20200104083057_7.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200104091152_3.png
Result_3/non_inclusions/TN02_20200104091152_3.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20200104100637_10.png
Result_3/inclusions/TN

...

Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201108013616_1.png
Result_3/inclusions/TN02_20201108013616_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201108015557_1.png
Result_3/inclusions/TN02_20201108015557_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201108023622_1.png
Result_3/non_inclusions/TN02_20201108023622_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201108033422_1.png
Result_3/inclusions/TN02_20201108033422_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201108042003_2.png
Result_3/inclusions/TN02_20201108042003_2.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201108044625_1.png
Result_3/inclusions/TN02_20201108044625_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201108061349_1.png
Result_3/non_inclusions/TN02_20201108061349_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201108070724_2.png
Result_3/inclusions/TN02_20201108070724_2.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201108090253_1.png
Result_3/non_inclusio

Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201112162027_2.png
Result_3/non_inclusions/TN02_20201112162027_2.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201112162027_3.png
Result_3/non_inclusions/TN02_20201112162027_3.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201112164319_1.png
Result_3/non_inclusions/TN02_20201112164319_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201112164319_2.png
Result_3/non_inclusions/TN02_20201112164319_2.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201112165745_1.png
Result_3/non_inclusions/TN02_20201112165745_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201112165745_2.png
Result_3/non_inclusions/TN02_20201112165745_2.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201112172548_1.png
Result_3/inclusions/TN02_20201112172548_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201112174843_1.png
Result_3/inclusions/TN02_20201112174843_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201112184307_1.png
Resul

Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115085750_6.png
Result_3/inclusions/TN02_20201115085750_6.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115085841_16.png
Result_3/inclusions/TN02_20201115085841_16.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115085841_23.png
Result_3/inclusions/TN02_20201115085841_23.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115085841_25.png
Result_3/inclusions/TN02_20201115085841_25.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115085841_26.png
Result_3/inclusions/TN02_20201115085841_26.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115085841_27.png
Result_3/inclusions/TN02_20201115085841_27.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115085841_28.png
Result_3/inclusions/TN02_20201115085841_28.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115085841_31.png
Result_3/inclusions/TN02_20201115085841_31.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115085841_4.png
Result_3/inclus

Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115092735_5.png
Result_3/inclusions/TN02_20201115092735_5.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115092826_12.png
Result_3/inclusions/TN02_20201115092826_12.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115092826_13.png
Result_3/inclusions/TN02_20201115092826_13.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115092826_16.png
Result_3/inclusions/TN02_20201115092826_16.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115092826_4.png
Result_3/inclusions/TN02_20201115092826_4.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115092826_5.png
Result_3/inclusions/TN02_20201115092826_5.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115092917_15.png
Result_3/inclusions/TN02_20201115092917_15.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115092917_16.png
Result_3/inclusions/TN02_20201115092917_16.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201115092917_17.png
Result_3/inclusion

Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201121082951_2.png
Result_3/non_inclusions/TN02_20201121082951_2.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201121141810_1.png
Result_3/non_inclusions/TN02_20201121141810_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201121145449_1.png
Result_3/non_inclusions/TN02_20201121145449_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201121145449_2.png
Result_3/non_inclusions/TN02_20201121145449_2.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201121152346_1.png
Result_3/non_inclusions/TN02_20201121152346_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201121152346_2.png
Result_3/non_inclusions/TN02_20201121152346_2.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201121205937_1.png
Result_3/non_inclusions/TN02_20201121205937_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201122002104_1.png
Result_3/non_inclusions/TN02_20201122002104_1.png
Unit Test Data/ALIS_UnitTest_2020_Q4/PNG/TN02_20201122051326_1.p

In [11]:
result = res[res['Type'] != res['Prediction']]
result

,Defect ID,Type,Prediction
114,TN02_20191125183647_2.png,inclusions,non_inclusions
265,TN02_20191125204915_2.png,inclusions,non_inclusions
807,TN02_20200101190903_1.png,inclusions,non_inclusions
909,TN02_20200103085854_1.png,inclusions,non_inclusions
927,TN02_20200103211424_17.png,non_inclusions,inclusions
...,...,...,...
8934,TN02_20201210093807_1.png,inclusions,non_inclusions
8943,TN02_20201212203138_1.png,inclusions,non_inclusions
8946,TN02_20201212220954_2.png,inclusions,non_inclusions
9012,TN02_20201223211330_2.png,inclusions,non_inclusions


In [12]:
result.shape

(139, 3)

In [13]:
type_1 = Index_table[Index_table['Defect ID'] =='TN02_20191125144955_3.png']['Type']
print(str(type_1))

0    BLISTER
Name: Type, dtype: object


In [14]:
import shutil
total_type=[]
path = 'Result_3'
img_path = 'Unit Test Data/ALIS_UnitTest_2020_Q4/PNG'
for i in range(len(result)):
    name = result.iloc[i]['Defect ID']
    folder = result.iloc[i]['Type']
    #print(img_path+"/"+str(name))
    #print(path+"/"+str(folder)+"/"+str(name))
    img= cv2.imread(img_path+"/"+str(name), 1)
    cv2.rectangle(img,( 250, 250), (750, 750)
                          , (0, 0, 255), 3)
    type_0=Index_table[Index_table['Defect ID'] ==str(name)]['Type'].iloc[0]
    total_type.append(type_0)
    print(type_0)
    name_0 = type_0+"_"+name
    print(name_0)
    print(path+"/failure/"+str(folder)+"/"+str(name_0))
    cv2.imwrite(path+"/failure/"+str(folder)+"/"+str(name_0), img)
    #shutil.copyfile(img_path+"/"+str(name), path+"/failure/"+str(folder)+"/"+str(name))


BLISTER
BLISTER_TN02_20191125183647_2.png
Result_3/failure/inclusions/BLISTER_TN02_20191125183647_2.png
BLISTER
BLISTER_TN02_20191125204915_2.png
Result_3/failure/inclusions/BLISTER_TN02_20191125204915_2.png
BLISTER
BLISTER_TN02_20200101190903_1.png
Result_3/failure/inclusions/BLISTER_TN02_20200101190903_1.png
INCLUSION_FLASHLIGHT
INCLUSION_FLASHLIGHT_TN02_20200103085854_1.png
Result_3/failure/inclusions/INCLUSION_FLASHLIGHT_TN02_20200103085854_1.png
DARKSPOT
DARKSPOT_TN02_20200103211424_17.png
Result_3/failure/non_inclusions/DARKSPOT_TN02_20200103211424_17.png
INCLUSION_FLASHLIGHT
INCLUSION_FLASHLIGHT_TN02_20200105140033_4.png
Result_3/failure/inclusions/INCLUSION_FLASHLIGHT_TN02_20200105140033_4.png
BROKENGLASS
BROKENGLASS_TN02_20200110061129_87.png
Result_3/failure/non_inclusions/BROKENGLASS_TN02_20200110061129_87.png
DARKSPOT
DARKSPOT_TN02_20200115055335_1.png
Result_3/failure/non_inclusions/DARKSPOT_TN02_20200115055335_1.png
BROKENGLASS
BROKENGLASS_TN02_20200116212230_158.png
Resu

SHEETEDGE
SHEETEDGE_TN02_20201020131613_1.png
Result_3/failure/non_inclusions/SHEETEDGE_TN02_20201020131613_1.png
INCLUSION_FLASHLIGHT
INCLUSION_FLASHLIGHT_TN02_20201021203202_24.png
Result_3/failure/inclusions/INCLUSION_FLASHLIGHT_TN02_20201021203202_24.png
SHEETEDGE
SHEETEDGE_TN02_20201021225334_2.png
Result_3/failure/non_inclusions/SHEETEDGE_TN02_20201021225334_2.png
INCLUSION_FLASHLIGHT
INCLUSION_FLASHLIGHT_TN02_20201022022225_11.png
Result_3/failure/inclusions/INCLUSION_FLASHLIGHT_TN02_20201022022225_11.png
SHEETEDGE
SHEETEDGE_TN02_20201022152016_1.png
Result_3/failure/non_inclusions/SHEETEDGE_TN02_20201022152016_1.png
SHEETEDGE
SHEETEDGE_TN02_20201022164653_2.png
Result_3/failure/non_inclusions/SHEETEDGE_TN02_20201022164653_2.png
CARRIER
CARRIER_TN02_20201022205538_1.png
Result_3/failure/non_inclusions/CARRIER_TN02_20201022205538_1.png
SHEETEDGE
SHEETEDGE_TN02_20201022211402_2.png
Result_3/failure/non_inclusions/SHEETEDGE_TN02_20201022211402_2.png
SHEETEDGE
SHEETEDGE_TN02_2020102

In [15]:
import shutil
total_type=[]
path = 'Result_2'
img_path = 'Unit Test Data/ALIS_UnitTest_2020_Q4/PNG'
for i in range(len(result)):
    name = result.iloc[i]['Defect ID']
    folder = result.iloc[i]['Type']
    #print(img_path+"/"+str(name))
    #print(path+"/"+str(folder)+"/"+str(name))
    img= cv2.imread(img_path+"/"+str(name), 1)

    type_0=Index_table[Index_table['Defect ID'] ==str(name)]['Type'].iloc[0]
    total_type.append(type_0)
    print(type_0)



    #shutil.copyfile(img_path+"/"+str(name), path+"/failure/"+str(folder)+"/"+str(name))


BLISTER
BLISTER
BLISTER
INCLUSION_FLASHLIGHT
DARKSPOT
INCLUSION_FLASHLIGHT
BROKENGLASS
DARKSPOT
BROKENGLASS
SHEETEDGE
BROKENGLASS
BROKENGLASS
BROKENGLASS
BROKENGLASS
BROKENGLASS
BROKENGLASS
BROKENGLASS
INCLUSION_FLASHLIGHT
BROKENGLASS
BROKENGLASS
BROKENGLASS
DARKSPOT
DARKSPOT
DARKSPOT
DARKSPOT
DARKSPOT
DARKSPOT
BROKENGLASS
DARKSPOT
DARKSPOT
DARKSPOT
DARKSPOT
DARKSPOT
DARKSPOT
DARKSPOT
DARKSPOT
DARKSPOT
DARKSPOT
DARKSPOT
DARKSPOT
DARKSPOT
DARKSPOT
DARKSPOT
INCLUSION_FLASHLIGHT
DARKSPOT
CARRIER
BROKENGLASS
SHEETEDGE
NO_DEFECT
DARKSPOT
NO_DEFECT
BLISTER
DARKSPOT
DARKSPOT
DARKSPOT
CARRIER
CARRIER
CARRIER
CARRIER
CARRIER
CARRIER
SHEETEDGE
INCLUSION_FLASHLIGHT
CARRIER
INCLUSION_SDX
INCLUSION_FLASHLIGHT
INCLUSION_FLASHLIGHT
SHEETEDGE
SHEETEDGE
SHEETEDGE
SHEETEDGE
SHEETEDGE
SHEETEDGE
SHEETEDGE
INCLUSION_FLASHLIGHT
SHEETEDGE
INCLUSION_FLASHLIGHT
SHEETEDGE
SHEETEDGE
CARRIER
SHEETEDGE
SHEETEDGE
SHEETEDGE
DARKSPOT
CARRIER
CARRIER
CARRIER
INCLUSION_FLASHLIGHT
CARRIER
CARRIER
CARRIER
CARRIER
BROKENG

In [16]:
result['families'] = total_type
result

c:\users\foresight\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Defect ID,Type,Prediction,families
114,TN02_20191125183647_2.png,inclusions,non_inclusions,BLISTER
265,TN02_20191125204915_2.png,inclusions,non_inclusions,BLISTER
807,TN02_20200101190903_1.png,inclusions,non_inclusions,BLISTER
909,TN02_20200103085854_1.png,inclusions,non_inclusions,INCLUSION_FLASHLIGHT
927,TN02_20200103211424_17.png,non_inclusions,inclusions,DARKSPOT
...,...,...,...,...
8934,TN02_20201210093807_1.png,inclusions,non_inclusions,BLISTER
8943,TN02_20201212203138_1.png,inclusions,non_inclusions,BLISTER
8946,TN02_20201212220954_2.png,inclusions,non_inclusions,BLISTER
9012,TN02_20201223211330_2.png,inclusions,non_inclusions,BLISTER


In [17]:
pd.value_counts(result[result['Type']=='inclusions']['families'])

BLISTER                 13
INCLUSION_FLASHLIGHT    11
INCLUSION_SDX            5
Name: families, dtype: int64

In [18]:
pd.value_counts(result[result['Type']=='non_inclusions']['families'])

DARKSPOT       46
CARRIER        20
BROKENGLASS    17
SHEETEDGE      16
NO_DEFECT      11
Name: families, dtype: int64